In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [2]:
%%capture
!pip install wandb --upgrade

In [3]:
import wandb
from wandb.keras import WandbCallback

In [4]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Subnetting train data and normalizing to [0., 1.]
x_train, x_test = x_train[::5] / 255., x_test / 255.
y_train = y_train[::5]

CLASS_NAMES = ["airplane", "automobile", "bird", "cat",
               "deer", "dog", "frog", "horse", "ship", "truck"]

170500096/170498071 [==============================] - 4s 0us/step


In [6]:
print("Shape of x_train: ", x_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_test: ", y_test.shape)

Shape of x_train:  (10000, 32, 32, 3)
Shape of y_train:  (10000, 1)
Shape of x_test : (10000, 32, 32, 3)
Shape of y_test:  (10000, 1)


In [12]:
def Model():
  inputs = keras.layers.Input(shape=(32, 32, 3))

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=2)(x)

  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)

  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Dense(32, activation='relu')(x)

  outputs = keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

In [8]:
# Initialize wandb with your project
run = wandb.init(project='my-keras-interation',
                 config={# and include hyperparameters and metadata
                         "learning_rate": 0.005,
                         "epochs": 5,
                         "batch_size": 64,
                         "loss_function": "sparse_categorical_crossentropy",
                         "architecture": "CNN",
                         "dataset": "CIFAR-10"})


config = wandb.config # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do
# Notice that we use config, so our metadata matches what gets executed
optimizer = tf.keras.optimizers.Adam(config.learning_rate)
model.compile(optimizer, config.loss_function, metrics=['acc'])

wandb: Currently logged in as: ajay-2007 (use `wandb login --relogin` to force relogin)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0     

In [9]:
# We train with our beloved model.fit
# Notice WandbCallback is used as a regular callback
# We again use config

_ = model.fit(x_train, y_train,
              epochs=config.epochs,
              batch_size=config.batch_size,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback()])

Epoch 1/5
157/157 [==============================] - 33s 12ms/step - loss: 2.1755 - acc: 0.1532 - val_loss: 2.0518 - val_acc: 0.2049
Epoch 2/5
157/157 [==============================] - 1s 8ms/step - loss: 2.0012 - acc: 0.2036 - val_loss: 1.9493 - val_acc: 0.2367
Epoch 3/5
157/157 [==============================] - 1s 9ms/step - loss: 1.8722 - acc: 0.2632 - val_loss: 1.7897 - val_acc: 0.3004
Epoch 4/5
157/157 [==============================] - 1s 7ms/step - loss: 1.7981 - acc: 0.3002 - val_loss: 1.9274 - val_acc: 0.2546
Epoch 5/5
157/157 [==============================] - 1s 7ms/step - loss: 1.7361 - acc: 0.3257 - val_loss: 1.7083 - val_acc: 0.3463


In [10]:
loss, accuracy = model.evaluate(x_test, y_test)

print("Test Error Rate: ", round((1-accuracy) * 100, 2))

# With wandb.log, we can easily pass in metrics as key-value pairs.
wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})

run.join()

313/313 [==============================] - 1s 3ms/step - loss: 1.7083 - acc: 0.3463
Test Error Rate:  65.37


epoch,4
loss,1.73608
acc,0.3257
val_loss,1.70826
val_acc,0.3463
_runtime,259
_timestamp,1629009643
_step,5
best_val_loss,1.70826
best_epoch,4
Test Error Rate,65.37


epoch,▁▃▅▆█
loss,█▅▃▂▁
acc,▁▃▅▇█
val_loss,█▆▃▅▁
val_acc,▁▃▆▃█
_runtime,▁▁▁▁▁█
_timestamp,▁▁▁▁▁█
_step,▁▂▄▅▇█
Test Error Rate,▁


In [13]:
# Initialize wandb with your project
run = wandb.init(project='my-keras-interation',
                 config={# and include hyperparameters and metadata
                         "learning_rate": 0.001,
                         "epochs": 5,
                         "batch_size": 32,
                         "loss_function": "sparse_categorical_crossentropy",
                         "architecture": "CNN",
                         "dataset": "CIFAR-10"})


config = wandb.config # We'll use this to configure our experiment

# Initialize model like you usually do.
tf.keras.backend.clear_session()
model = Model()
model.summary()

# Compile model like you usually do
# Notice that we use config, so our metadata matches what gets executed
optimizer = tf.keras.optimizers.Adam(config.learning_rate)
model.compile(optimizer, config.loss_function, metrics=['acc'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0     

In [14]:
# We focus on a subset of images, since this is for human review

val_images, val_labels = x_test[:32], y_test[:32]

# Our beloved model.fit returns
# By passing arguments to WandbCallback, we change its behavior

_ = model.fit(x_train, y_train,
              epochs=config.epochs,
              validation_data=(x_test, y_test),
              callbacks=[WandbCallback(data_type='image',
                                       training_data=(val_images, val_labels),
                                       labels=CLASS_NAMES)])

Epoch 1/5
313/313 [==============================] - 3s 7ms/step - loss: 2.1153 - acc: 0.1730 - val_loss: 1.9116 - val_acc: 0.2201
Epoch 2/5
313/313 [==============================] - 3s 9ms/step - loss: 1.8468 - acc: 0.2754 - val_loss: 1.7608 - val_acc: 0.2998
Epoch 3/5
313/313 [==============================] - 3s 8ms/step - loss: 1.7061 - acc: 0.3270 - val_loss: 1.7222 - val_acc: 0.3236
Epoch 4/5
313/313 [==============================] - 3s 8ms/step - loss: 1.6473 - acc: 0.3652 - val_loss: 1.6054 - val_acc: 0.3829
Epoch 5/5
313/313 [==============================] - 3s 9ms/step - loss: 1.5801 - acc: 0.3984 - val_loss: 1.5228 - val_acc: 0.4258
